<a href="https://colab.research.google.com/github/d1p013/testing/blob/master/Simple_Search_Engine_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI6122 Assignment 1 - Section 3.3


## Prerequisites

You will need PyTerrier installed. PyTerrier also needs Java to be installed, and will find most installations.

In [9]:
!pip install python-terrier
#!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier

## Init 

You must run `pt.init()` before other pyterrier functions and classes

Optional Arguments:    
 - `version` - terrier IR version e.g. "5.2"    
 - `mem` - megabytes allocated to java e.g. "4096"      
 - `packages` - external java packages for Terrier to load e.g. ["org.terrier:terrier.prf"]
 - `logging` - logging level for Terrier. Defaults to "WARN", use "INFO" or "DEBUG" for more output.

NB: PyTerrier needs Java 11 installed. If it cannot find your Java installation, you can set the `JAVA_HOME` environment variable.

In [10]:
import pyterrier as pt
if not pt.started():
  pt.init()

 ## Importing dataset from Google Drive

Using built-in function of Google Colab, we can easily import the dataset which has been uploaded onto Google Drive beforehand

need a Google account to download the data

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID which is residing on Google Drive
#
file_id = '1aVXMJ_luTXISxMwP5_Bt2xE0ObXqEsQ2' #"Dataset_B1to8"

downloaded = drive.CreateFile({'id': file_id})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))
downloaded.GetContentFile('Dataset_B1to8.csv')

## Loading dataset (csv) into Pandas dataframe

PyTerrier makes it easy to index standard Python data structures, particularly [Pandas dataframes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

importing dataset, which is scale down version of the original `review.json` file as the original file is too large to load into Google Colab memory


In [ ]:
## load data into df
import pandas as pd
#df = pd.read_csv("AI6122_Dataset_B1.csv", dtype = str)
df = pd.read_csv("Dataset_B1to8.csv", dtype = str)

try :
  del docno 
except:
  pass

docno =[]
for idx in range(1,len(df)+1):
  docno.append("d" + str(idx))

df["docno"] = docno


## Time taken for indexing

Tracking the time to complete indexing for each 10% incremental of documents. The index folder is cleared before each indexing and time and is taken before and after indexing, the difference being the time taken to index


In [ ]:
import time

Tcollect = []
for idx in range(1,11):
  #print(idx)
  df_resize = df.iloc[:int(len(df)/(10/idx)),:]
  !rm -rf ./pd_index
  pd_indexer = pt.DFIndexer("./pd_index", overwrite=True, verbose=True)
  Tstart = time.perf_counter()
  indexref = pd_indexer.index(df_resize["text"], df_resize)
  Tend = time.perf_counter()
  print(str(10*idx)+f"% df search completed in {Tend - Tstart:0.4f} seconds")
  Tcollect.append(Tend - Tstart)



In [ ]:
import matplotlib.pyplot as plt

plotdf = pd.DataFrame({
    'Percentage Completed':['10%', '20%', '30%', '40%', '50%', 
                            '60%', '70%', '80%', '90%', '100%'],
    'Time(s)':Tcollect
    })

# a scatter plot comparing num_children and num_pets
plotdf.plot(kind='line',x='Percentage Completed',y='Time(s)',color='blue')
plt.show()


## Indexing a Pandas dataframe

We can use a `pt.DFIndexer()` object to do indexing for Pandas dataframe

In [ ]:
#import pandas as pd
!rm -rf ./pd_index
pd_indexer = pt.DFIndexer("./pd_index", overwrite=True, verbose=True, blocks=True)

# optionally modify properties
# index_properies = {"block.indexing":"true", "invertedfile.lexiconscanner":"pointers"}
# indexer.setProperties(**index_properies)
pd_indexer.setProperty("tokeniser" , "EnglishTokeniser")
pd_indexer.setProperty("termpipelines", "Stopwords,PorterStemmer")
#pd_indexer.setProperty("block.indexing","true")
#pd_indexer.setProperty("termpipelines", "Stopwords")
# no stemming or stopwords
#pd_indexer.setProperty("termpipelines", "")

Then there are a number of options to index the dataframe:    
The first argument should always a pandas.Series object of Strings, which specifies the body of each document.    
Any arguments after that are for specifying metadata.

We can view more useful information from the indexed objects using a indexref.getCollectionsStatistics()

In [ ]:
import time

# Add metadata fields as Pandas.Series objects, with the name of the Series object becoming the name of the meta field.
#indexref = pd_indexer.index(df["text"], df["docno"], df["review_id"], df["user_id"], df["business_id"], df["stars"], df["useful"], df["funny"], df["cool"])
Tstart = time.perf_counter()
# Add the entire dataframe as metadata
indexref = pd_indexer.index(df["text"], df)
Tend = time.perf_counter()
print(f"search completed in {Tend - Tstart:0.4f} seconds")
indexinfo = pt.IndexFactory.of(indexref)
print(indexinfo.getCollectionStatistics().toString()) 

  0%|          | 0/6928 [00:00<?, ?documents/s]

search completed in 10.6397 seconds
Number of documents: 6928
Number of terms: 12049
Number of postings: 323743
Number of fields: 0
Number of tokens: 388543
Field names: []
Positions:   true



In the above example, the indexed collection had 6928 documents, which contained 388543-word occurrences. Out of which 12049 were identified as unique words. The total postings in the inverted index are 323743. 

The whole datafame is being index, with the `"text"` field being searchable while the remaining (e.g. `"review_id"`, `"user_id"`, `"business_id"`, `"stars"`, `"date"`, `"useful"`, `"funny"`, `"cool"`, `"text"`) as metadata which can be displayed when called upon.

pyTerrier perform standard stopwords removal and applies Porter's stemmer by default, and it is applicable in this notebook as well.
EnglishTokeniser is the default tokeniser and case-folding to lower case is applied during Tokenization

## Retrieval

Retrieval takes place using the `BatchRetrieve` object, by invoking `transform()` method for one or more queries. For a quick test, you can give just pass your query to `transform()`. 

BatchRetrieve will return the results as a Pandas dataframe.


In [ ]:
#pt.BatchRetrieve(indexref).search("he")

In [ ]:

#this ranker will make the candidate set of documents for each query
#BM25 = pt.BatchRetrieve(indexref, controls = {"wmodel": "BM25"}, num_results=5)
#these rankers we will use to re-rank the BM25 results
#TF_IDF = pt.BatchRetrieve(indexref, controls = {"wmodel": "TF_IDF"}, num_results=5)
#PL2 =  pt.BatchRetrieve(indexref, controls = {"wmodel": "PL2"}, num_results=5)

#pipe = BM25 >> (TF_IDF ** PL2)
#pipe.transform("really cute restaurant") 


In [ ]:
#pt.BatchRetrieve(indexref, controls = {"wmodel": "PL2"}, num_results=5).search("Really cute restaurant")


In [ ]:
#pt.BatchRetrieve(indexref, wmodel="BM25", properties={"termpipelines" : "Stopwords,PorterStemmer"})
#pt.BatchRetrieve(indexref, controls = {"wmodel": "BM25"}, properties={"termpipelines" : ""}, num_results=10).search("Really cute restaurant")
#pt.BatchRetrieve(indexref, metadata=["business_id", "stars"], num_results=10).search("Really cute restaurant")


However, most IR experiments, will use a set of queries. You can pass such a set using a data frame for input.

In [ ]:
#import pandas as pd
#topics = pd.DataFrame([["q1", "Really cute restaurant"], ["q2", "restaurant"]],columns=['qid','query'])
#pt.BatchRetrieve(indexref, metadata=["text"], num_results=5).transform(topics)

## Simple Search Engine

Prompting user input for search string
Prompting user input for the Top N results to return
Display the time taken to complete the search
Display the search results

# Definition for Simple Search Engine

In [ ]:
def SimpleSearch():
  import time
  import pandas as pd
  pd.reset_option('^display.', silent=True)
  #pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)
  #pd.set_option('display.width', None)
  #pd.set_option('display.max_colwidth', -1)

  search_str = input("Please enter your search string: ")
  #print("Search string: ", search_str)
  TopN = input("Please enter number results to display: ")
  #print("Top N results: ", TopN)
  Tstart = time.perf_counter()
  topics = pd.DataFrame([["q1", search_str]],columns=['qid','query'])
  results = pt.BatchRetrieve(indexref, wmodel="TF_IDF", properties={"termpipelines" : "Stopwords,PorterStemmer"}, metadata=["text"], num_results=int(TopN)).transform(topics)
  Tend = time.perf_counter()
  print(f"search completed in {Tend - Tstart:0.4f} seconds")

  if len(results) == 0:
    print("no result found!")
  #else:
    #print("len: ", len(results))
    #print(results)

  return results

# Search #1
Keywords "restaurant"

In [ ]:
SimpleSearch()

Please enter your search string: quality restaurant
Please enter number results to display: 5
search completed in 0.0977 seconds


,qid,docid,text,rank,score,query
0,q1,4389,Amazing!!! Stumbled upon it on an adventure t...,0,5.353792,quality restaurant
1,q1,1845,I used to like this restaurant but the quality...,1,5.267997,quality restaurant
2,q1,5405,A quality restaurant with excellent food. The ...,2,5.214978,quality restaurant
3,q1,1650,Lovely servers with good quality Greek food.In...,3,5.210742,quality restaurant
4,q1,1817,Awful and rude customer service. Food quality ...,4,5.210742,quality restaurant


# Search #2
Keywords "honda"

In [ ]:
SimpleSearch()

Please enter your search string: honda
Please enter number results to display: 5
search completed in 0.0464 seconds
no result found!


,docid,text,rank,score,qid,query


#Search #3
Key phrase "homemade pasta"
rank 5 dont have the search phrase but got the terms

In [ ]:
SimpleSearch()

Please enter your search string: ribeye pasta
Please enter number results to display: 5
search completed in 0.0592 seconds


,qid,docid,text,rank,score,query
0,q1,4291,Great meal! Started with the tuna tartar whic...,0,8.410616,ribeye pasta
1,q1,4142,Another wannabe steakhouse that cannot get the...,1,8.168902,ribeye pasta
2,q1,4089,Never disappoints. Attentive service and every...,2,7.406424,ribeye pasta
3,q1,4093,We have visited L'Andana in several occasions ...,3,6.811879,ribeye pasta
4,q1,4332,"Pam, the bartender, makes a perfect cosmo. Bl...",4,5.911969,ribeye pasta


#trial

In [15]:
# define an example dataframe of documents
import pandas as pd
!rm -rf ./trial_index
trial_df = pd.DataFrame({
    'docno':
    ['1', '2', '3'],
    'url':
    ['url1', 'url2', 'url3'],
    'text':
    ['He ran out of money, so he had to stop playing cook the loss',
    'He The waves were crashing on the shore; it was a, cooks',
    'The body may perhaps compensates for the loss, cooking waves']
})

#trial_df = pd.read_csv("small_B1_edit.csv", dtype = str)
#trial_df

# index the text, record the docnos as metadata
trial_indexer = pt.DFIndexer("./trial_index", blocks=True)
trial_indexer.setProperty("tokeniser" , "EnglishTokeniser")
#trial_indexer.setProperty("termpipelines" , "Stopwords,PorterStemmer")
trial_indexer.setProperty("termpipelines" , "")

trial_indexref = trial_indexer.index(trial_df["text"], trial_df["docno"])
trial_indexinfo = pt.IndexFactory.of(trial_indexref)
print(trial_indexinfo.getCollectionStatistics().toString()) 

topics = pd.DataFrame([["q1", "'the '~1"]],columns=['qid','query'])
pt.rewrite.reset()
sdm = pt.rewrite.SequentialDependence()
#sdm = pt.rewrite.SequentialDependence(prox_model="org.terrier.matching.models.Dirichlet_LM")
#pt.BatchRetrieve(trial_indexref, properties={"termpipelines" :"Stopwords,PorterStemmer"}, controls={"wmodel" : "TF_IDF"}).search("cook")
dph = pt.BatchRetrieve(trial_indexref, wmodel="dph")
pipeline = sdm >> dph
pipeline.transform(topics)

Number of documents: 3
Number of terms: 28
Number of postings: 33
Number of fields: 0
Number of tokens: 36
Field names: []
Positions:   true



JavaException: ignored

@inproceedings{pyterrier2020ictir,

    author = {Craig Macdonald and Nicola Tonellotto},
    title = {Declarative Experimentation inInformation Retrieval using PyTerrier},
    booktitle = {Proceedings of ICTIR 2020},
    year = {2020}
}

